In [ ]:
library(WGCNA)
library(qs2)
library(tidyverse)
options(stringsAsFactors = FALSE)

# https://space.bilibili.com/3546800304687494

In [ ]:
# 创建目录（若存在则静默跳过）
fs::dir_create("WGCNA_output", recurse = TRUE)

WGCNA(加权共表达网络分析):寻找协同表达的基因模块，探索基因网络与关注的表型之间的关联关系，以及网络中的核心（hub）基因

计算每一对基因直接的表达相关性，将具有相似表达模式的基因归类到同一个模块当中，那么这些模块当中的基因可能共同参与调控某个生物学过程

通过网络的方式展示基因之间的关系，每个基因被视为一个节点，之间的关系用边来表示，通过分析得到网络当中密切联系的基因群体也就是模块还可找到模块中最具连接性和调控潜力的关键基因也就是核心基因。不仅可以找到与表型相关的模块，还能进一步挖掘出模块当中可能具有生物学意义的基因

基因表达矩阵 -> 相关性矩阵 -> 邻接矩阵

基因表达矩阵计算相关性后转换为相关性矩阵，那么相关性多大才算两个基因相关呢？

相关性矩阵以软阈值取幂次后称为邻接矩阵，通过图片去进行软阈值的挑选

识别基因所在模块的时候，把基因所在的圈子也考虑在内，使得 邻接矩阵 -> 拓扑矩阵

拓扑矩阵 -> 识别基因模块

总结：WGCNA根据基因表达相关性，把基因归类到模块，将模块与生物学功能联系起来，探索模块可能具有的生物学意义，从而探索生物标志物，治疗靶点，未知基因功能等

# 表达矩阵清洗

In [49]:
load("../output_mRNA_lncRNA_expr/TCGA-LUAD_mrna_expr_fpkm.rdata")

In [ ]:
# 挑选基因，采用绝对中位差mad过滤数据
mrna_exp_mad = apply(mrna_expr_fpkm, 1, mad)
mrna_exp_mad = order(mrna_exp_mad, decreasing = TRUE)
mrna_exp_num = mrna_exp_mad[1:5000]
mrna_exp_filter = mrna_expr_tpm[mrna_exp_num,]
datExpr0 = t(mrna_exp_filter)

In [43]:
# 对样本和基因进行评估，剔除低质量的样本和基因
gsg = goodSamplesGenes(datExpr0, verbose = 3) 
gsg$allOK
if (!gsg$allOK)
{
  if (sum(!gsg$goodGenes)>0)
    printFlush(paste("Removing genes:", paste(names(datExpr0)[!gsg$goodGenes], collapse = ", ")))
  if (sum(!gsg$goodSamples)>0)
    printFlush(paste("Removing samples:", paste(rownames(datExpr0)[!gsg$goodSamples], collapse = ", ")))
  datExpr0 = datExpr0[gsg$goodSamples, gsg$goodGenes]
}

 Flagging genes and samples with too many missing values...
  ..step 1


In [ ]:
# 聚类树，对离群样本进行剔除
sampleTree = hclust(dist(datExpr0), method = "average")
pdf(file = paste0("WGCNA_output/", "1_sampleClustering.pdf"), width = 25, height = 12)
par(cex = 0.6)
par(mar = c(0,4,2,0))
plot(sampleTree, main = "Sample clustering to detect outliers", sub="", xlab="", cex.lab = 1.5,
     cex.axis = 1.5, cex.main = 2)

# Plot a line to show the cut
abline(h = 110000, col = "red")# 样本剪切高度，在这个高度正好把离群样本和其他样本分割开来
dev.off()

pdf 
  2 

In [ ]:
#选做，根据高度过滤样本
# Determine cluster under the line
clust = cutreeStatic(sampleTree, cutHeight = 110000, minSize = 10)
print(table(clust)) # 0表示要剔除的样本，1表示要保留的样本

# clust 1 contains the samples we want to keep.
keepSamples = (clust==1)
datExpr0 = datExpr0[keepSamples, ]

clust
  0   1 
  4 596 


In [52]:
qs_save(datExpr0, "WGCNA_output/dataExpr0.qs2")

In [9]:
rm(list = ls())
gc()

           used  (Mb) gc trigger  (Mb) max used  (Mb)
Ncells  5992451 320.1   10629622 567.7  9254636 494.3
Vcells 41012808 313.0   63386204 483.6 52754596 402.5

# 表型信息处理

In [ ]:
traitdata = qs_read("bp_res/bp_res_fraction.qs2") |> as.data.frame()
datExpr0 = qs_read("WGCNA_output/dataExpr0.qs2")

In [71]:
traitdata = traitdata |> select(pDC.c_Polarized, pDC.e_Polarized)

In [75]:
traitdata = traitdata |> 
  mutate(tumor = tinyarray::make_tcga_group(mrna_expr_fpkm)) |> 
  mutate(tumor = ifelse(tumor == "tumor", 1, 0))

In [ ]:
# 对齐表型数据和表达数据的样本
# 获取共有的样本名，并按表达数据的顺序排列
common_samples <- intersect(rownames(datExpr0), rownames(traitdata))
# 过滤表达数据和性状数据，仅保留共有样本，并确保顺序一致
datExpr0 <- datExpr0[common_samples, , drop = FALSE]
traitdata <- traitdata[common_samples, , drop = FALSE]

In [79]:
# Re-cluster samples
sampleTree2 = hclust(dist(datExpr0), method = "average")
traitColors = numbers2colors(traitdata, signed = FALSE)
#sizeGrWindow(12,12)
pdf(file=paste0("WGCNA_output/", "2_Sample dendrogram and trait heatmap.pdf"),width=25,height=10)
plotDendroAndColors(sampleTree2, traitColors,
                    groupLabels = names(traitdata),
                    main = "Sample dendrogram and trait heatmap",marAll = c(3,10,3,3))
dev.off()

pdf 
  2 

In [80]:
qs_save(traitdata, "WGCNA_output/traitdata.qs2")

In [163]:
rm(list = ls())
gc()

           used  (Mb) gc trigger  (Mb)  max used   (Mb)
Ncells  6865853 366.7   10629622 567.7  10629622  567.7
Vcells 17770134 135.6   64705289 493.7 157971835 1205.3

# 逐步法识别基因模块

In [166]:
traitdata = qs_read("WGCNA_output/datTraits.qs2")
datExpr0 = qs_read("WGCNA_output/datExpr0.qs2")

In [167]:
# 多线程
enableWGCNAThreads(nThreads = 8)  

Allowing parallel execution with up to 8 working processes.


In [85]:
# Choose a set of soft-thresholding powers
powers = c(1:20)

# Call the network topology analysis function
sft = pickSoftThreshold(datExpr0, powerVector = powers, verbose = 5)

pickSoftThreshold: will use block size 5000.
 pickSoftThreshold: calculating connectivity for given powers...
   ..working on genes 1 through 5000 of 5000
   Power SFT.R.sq  slope truncated.R.sq  mean.k. median.k.  max.k.
1      1    0.106  0.936          0.760 746.0000  7.54e+02 1230.00
2      2    0.151 -0.780          0.895 183.0000  1.79e+02  444.00
3      3    0.485 -1.520          0.959  58.4000  5.32e+01  194.00
4      4    0.641 -1.820          0.973  22.3000  1.85e+01   96.60
5      5    0.733 -1.870          0.972   9.7700  7.22e+00   51.90
6      6    0.764 -2.140          0.967   4.7500  3.06e+00   33.80
7      7    0.833 -2.200          0.984   2.5200  1.40e+00   24.60
8      8    0.870 -2.160          0.976   1.4400  6.80e-01   18.50
9      9    0.893 -2.090          0.969   0.8770  3.50e-01   14.40
10    10    0.906 -2.000          0.962   0.5630  1.85e-01   11.40
11    11    0.910 -1.920          0.955   0.3780  1.00e-01    9.23
12    12    0.926 -1.810          0.953  

In [86]:
# Plot the results:
#sizeGrWindow(9, 5)
pdf(file=paste0("WGCNA_output/", "3_Scale independence.pdf"),width=9,height=5)
par(mfrow = c(1,2))
cex1 = 0.9
# Scale-free topology fit index as a function of the soft-thresholding power
plot(sft$fitIndices[,1], -sign(sft$fitIndices[,3])*sft$fitIndices[,2],
     xlab="Soft Threshold (power)",ylab="Scale Free Topology Model Fit,signed R^2",type="n",
     main = paste("Scale independence"));
text(sft$fitIndices[,1], -sign(sft$fitIndices[,3])*sft$fitIndices[,2],
     labels=powers,cex=cex1,col="red");
# this line corresponds to using an R^2 cut-off of h
abline(h=0.9,col="red")
# Mean connectivity as a function of the soft-thresholding power
plot(sft$fitIndices[,1], sft$fitIndices[,5],
     xlab="Soft Threshold (power)",ylab="Mean Connectivity", type="n",
     main = paste("Mean connectivity"))
text(sft$fitIndices[,1], sft$fitIndices[,5], labels=powers, cex=cex1,col="red")
dev.off()

pdf 
  2 

In [91]:
#chose the softPower
softPower =sft$powerEstimate # 提取最佳软阈值
print(sft$powerEstimate)
adjacency = adjacency(datExpr0, power = softPower) # 基因表达矩阵 -> 相关性矩阵 -> 邻接矩阵

[1] 8


In [ ]:
k = as.vector(apply(adjacency, 2, sum, na.rm=T))

In [ ]:
par(mfrow = c(1,2))
cex = 0.9
hist(k)
scaleFreePlot(k, main = "check scale free topology") # 注意R^2是否大于0.85

  scaleFreeRsquared slope
1              0.92 -3.01

In [96]:
# Turn adjacency into topological overlap
TOM = TOMsimilarity(adjacency) # 邻接矩阵 -> 拓扑矩阵
dissTOM = 1-TOM # 计算基因相异度

..connectivity..
..matrix multiplication (system BLAS)..
..normalization..
..done.


In [ ]:
# Call the hierarchical clustering function
geneTree = hclust(as.dist(dissTOM), method = "average");
# Plot the resulting clustering tree (dendrogram)

#sizeGrWindow(12,9)
pdf(file=paste0("WGCNA_output/", "4_Gene clustering on TOM-based dissimilarity.pdf"),width=12,height=9)
plot(geneTree, xlab="", sub="", main = "Gene clustering on TOM-based dissimilarity",
     labels = FALSE, hang = 0.04)
dev.off()

[1] 8

In [ ]:
# 动态剪切树逐步识别网络模块
# We like large modules, so we set the minimum module size relatively high:
minModuleSize = 30
# Module identification using dynamic tree cut:
dynamicMods = cutreeDynamic(dendro = geneTree, # 基因聚类结果
                            distM = dissTOM, # 基因相异度
                            deepSplit = 2, # 调整划分模块的敏感度，数值（0-4）越大越敏感，得到的模块越多
                            pamRespectsDendro = FALSE, # 不对相似度高的模块进行自动合并
                            minClusterSize = minModuleSize)  # 模块包含的最小基因数量，数值越小，小的模块会被保留，模块越多
table(dynamicMods) # 模块数和每个模块所含的基因数

 ..cutHeight not given, setting it to 0.999  ===>  99% of the (truncated) height range in dendro.
 ..done.


dynamicMods
   0    1    2    3    4    5    6    7    8    9   10   11   12   13   14   15   16   17   18 
1419  625  521  419  341  329  178  169  148  138  108   99   91   86   75   73   66   59   56 

In [ ]:
# 绘制基因聚类树图
# Convert numeric lables into colors
dynamicColors = labels2colors(dynamicMods)
table(dynamicColors)
# Plot the dendrogram and colors underneath
#sizeGrWindow(8,6)
pdf(file=paste0("WGCNA_output/", "5_Dynamic Tree Cut.pdf"),width=8,height=6)
plotDendroAndColors(geneTree, dynamicColors, "Dynamic Tree Cut",
                    dendroLabels = FALSE, hang = 0.03,
                    addGuide = TRUE, guideHang = 0.05,
                    main = "Gene dendrogram and module colors")
dev.off()

pdf 
  2 

In [101]:
# Calculate eigengenes,计算每个模块的特征基因表达量
MEList = moduleEigengenes(datExpr0, colors = dynamicColors)
MEs = MEList$eigengenes # 每个基因模块在每个样本中的表达量
# Calculate dissimilarity of module eigengenes，计算模块特征基因相异度
MEDiss = 1-cor(MEs);
# Cluster module eigengenes，根据相似度进行聚类
METree = hclust(as.dist(MEDiss), method = "average")
# Plot the result
#sizeGrWindow(7, 6)
pdf(file=paste0("WGCNA_output/", "6_Clustering of module eigengenes.pdf"),width=7,height=6)
plot(METree, main = "Clustering of module eigengenes",
     xlab = "", sub = "")
MEDissThres = 0.25 # 剪切高度可修改，低于高度的模块可以进行合并
# Plot the cut line into the dendrogram
abline(h=MEDissThres, col = "red")
dev.off()

pdf 
  2 

In [ ]:
# Call an automatic merging function，合并相异度较低的模块
merge = mergeCloseModules(datExpr0, dynamicColors, cutHeight = MEDissThres, verbose = 3)
# The merged module colors，提取合并后模块对应的颜色
mergedColors = merge$colors
print(length(table(mergedColors))) # 查看模块数量
# Eigengenes of the new merged modules，提取合并之后每个基因模块的表达量
mergedMEs = merge$newMEs

 mergeCloseModules: Merging modules whose distance is less than 0.25
   multiSetMEs: Calculating module MEs.
     Working on set 1 ...
     moduleEigengenes: Calculating 19 module eigengenes in given set.
   multiSetMEs: Calculating module MEs.
     Working on set 1 ...
     moduleEigengenes: Calculating 16 module eigengenes in given set.
   Calculating new MEs...
   multiSetMEs: Calculating module MEs.
     Working on set 1 ...
     moduleEigengenes: Calculating 16 module eigengenes in given set.


In [105]:
# 可视化合并之后的网络模块
#sizeGrWindow(12, 9)
pdf(file=paste0("WGCNA_output/", "7_merged dynamic.pdf"), width = 9, height = 6.5)
plotDendroAndColors(geneTree, cbind(dynamicColors, mergedColors),
                    c("Dynamic Tree Cut", "Merged dynamic"),
                    dendroLabels = FALSE, hang = 0.03,
                    addGuide = TRUE, guideHang = 0.05)
dev.off()

pdf 
  2 

In [106]:
# Rename to moduleColors
moduleColors = mergedColors
# Construct numerical labels corresponding to the colors
colorOrder = c("grey", standardColors(50))
moduleLabels = match(moduleColors, colorOrder)-1
MEs = mergedMEs
save(softPower, MEs, moduleLabels, moduleColors, geneTree, file = "WGCNA_output/逐步法识别基因模块.rdata")

In [ ]:
rm(list = ls())
gc()

# 表型信息关联分析

In [169]:
datExpr0 = qs_read("WGCNA_output/datExpr0.qs2")
datTraits = qs_read("WGCNA_output/datTraits.qs2")
load("WGCNA_output/逐步法识别基因模块.rdata")

In [170]:
# 对模块特征基因表达矩阵进行排序，使相似的基因模块（通过相关性测量）相邻
MEs = orderMEs(MEs)
names = rownames(MEs)
datTraits = datTraits[names,] # 样本对其
nGenes = ncol(datExpr0)
nSamples = nrow(datExpr0)
# 计算模块和表型信息相关性
moduleTraitCor = cor(MEs, datTraits, use = "p") # 行名是模块名，列名是表型信息
# 计算模块和表型信息相关性p值
moduleTraitPvalue = corPvalueStudent(moduleTraitCor, nSamples)

In [171]:
#sizeGrWindow(10,6)
pdf(file=paste0("WGCNA_output/", "8_Module-trait relationships.pdf"),width=9,height=8)
# Will display correlations and their p-values
textMatrix = paste(signif(moduleTraitCor, 2), "\n(",
                   signif(moduleTraitPvalue, 1), ")", sep = "")

dim(textMatrix) = dim(moduleTraitCor)
par(mar = c(10, 10, 5, 5))

# Display the correlation values within a heatmap plot
labeledHeatmap(Matrix = moduleTraitCor,
  xLabels = names(datTraits),
  yLabels = names(MEs),
  ySymbols = names(MEs),
  colorLabels = FALSE,
  colors = blueWhiteRed(50),
  textMatrix = textMatrix,
  setStdMargins = FALSE,
  cex.text = 0.5,font.lab.x = 0.5,font.lab.y = 0.5,
  zlim = c(-1,1),
  main = paste("Module-trait relationships"))
dev.off()

# 可以把相关性最高的模块提取出来做最后的分析

In [172]:
# names (colors) of the modules
modNames = substring(names(MEs), 3) #提取模块颜色，从第三个字符开始

geneModuleMembership = as.data.frame(cor(datExpr0, MEs, use = "p")) # 计算各个基因和模块间的相关性
MMPvalue = as.data.frame(corPvalueStudent(as.matrix(geneModuleMembership), nSamples)) # 计算相关性p值

# 修改列名,重新添加MM
names(geneModuleMembership) = paste("MM", modNames, sep="")
names(MMPvalue) = paste("p.MM", modNames, sep="")

#names of those trait
traitNames=names(datTraits)

geneTraitSignificance = as.data.frame(cor(datExpr0, datTraits, use = "p"))
GSPvalue = as.data.frame(corPvalueStudent(as.matrix(geneTraitSignificance), nSamples))

names(geneTraitSignificance) = paste("GS.", traitNames, sep="")
names(GSPvalue) = paste("p.GS.", traitNames, sep="")


In [173]:
for (trait in traitNames){
  traitColumn=match(trait,traitNames)
  
  for (module in modNames){
    column = match(module, modNames) # 模块特征基因所在列
    moduleGenes = moduleColors==module # 提取某个模块中的基因
    
    # 基因与模块相关性--基因与性状相关性 散点图，这个基因是否在性状和模块当中都扮演重要角色？（靠右上方）
    if (nrow(geneModuleMembership[moduleGenes,]) > 1){#进行这部分计算必须每个模块内基因数量大于2，由于前面设置了最小数量是30，这里可以不做这个判断，但是grey有可能会出现1个gene,它会导致代码运行的时候中断，故设置这一步
      
      #sizeGrWindow(7, 7)
      pdf(file=paste0("WGCNA_output/", paste("9_", trait, "_", module,"_Module membership vs gene significance.pdf",sep="")),width=7,height=7)
      par(mfrow = c(1,1))
      verboseScatterplot(abs(geneModuleMembership[moduleGenes, column]),
                         abs(geneTraitSignificance[moduleGenes, traitColumn]),
                         xlab = paste("Module Membership in", module, "module"),
                         ylab = paste("Gene significance for ",trait),
                         main = paste("Module membership vs. gene significance\n"),
                         cex.main = 1.2, cex.lab = 1.2, cex.axis = 1.2, col = module)
      dev.off()
    }
  }
}

In [174]:
# 感兴趣性状和所有模块之间的相关性
for (trait in colnames(geneTraitSignificance)) {
  # 使用双括号提取动态列名
  GS = geneTraitSignificance[[trait]] 
  GS = abs(GS)
  
  pdf(file = paste0("WGCNA_output/10_Gene Significance for ", trait, " across module.pdf"))
  
  plotModuleSignificance(
    GS, 
    moduleColors,
    ylim = c(0, 0.2),
    main = paste0("WGCNA_output/10_Gene Significance for ", trait, " across module")
  )

  dev.off() # 关闭图形设备
}

In [175]:
# 提取每个模块hub基因
hubgenes_of_modules = chooseTopHubInEachModule(datExpr0, moduleColors) 
hubgenes_of_modules = hubgenes_of_modules|> as.data.frame() |> rownames_to_column("moduleColors")
write_csv(hubgenes_of_modules, paste0("WGCNA_output/", "11_hubgenes_of_modules.csv"))

In [ ]:
# 筛选感兴趣性状相关的基因

for (trait in colnames(datTraits)) {
  trait_related_genes = networkScreening(
    datTraits[[trait]],
    MEs,
    datExpr0
  )
  
  trait_related_genes = as.data.frame(trait_related_genes) |>
    rownames_to_column("gene_symbol") |>
    arrange(desc(cor.Weighted))  # 按 cor.Weighted 降序排列
  
  write_csv(
    trait_related_genes,
    paste0("WGCNA_output/11_", trait, "_related_genes.csv")  # 文件名添加分隔符 _
  )
}

# 网络模块可视化

In [3]:
datExpr0 = qs_read("WGCNA_output/datExpr0.qs2")
datTraits = qs_read("WGCNA_output/datTraits.qs2")
load("WGCNA_output/逐步法识别基因模块.rdata")

In [6]:
# 提取感兴趣模块中的基因
modules = c("brown", "green")
inModule = (moduleColors==modules) # 基因所在位置
modGenes = colnames(datExpr0)[inModule]

# 计算拓扑矩阵,并提取对应模块的拓扑矩阵
TOM = TOMsimilarityFromExpr(datExpr0, power = softPower)
modTOM = TOM[inModule, inModule]

# 输出为ctyoscape可识别的格式
cyt = exportNetworkToCytoscape(
  modTOM,
  nodeFile = paste("WGCNA_output/", "CytoscapeInput-nodes-", paste(modules, collapse = "-")),
  edgeFile = paste("WGCNA_output/", "CytoscapeInput-edges-", paste(modules, collapse = "-")),
  weighted = TRUE, # 是否为加权网络
  threshold = 0.02, # 在邻接矩阵中高于多少定义为相关
  nodeNames = modGenes, # 节点名字=基因名
  nodeAttr = moduleColors[inModule] # 节点属性=颜色
)

TOM calculation: adjacency..
..will not use multithreading.
 Fraction of slow calculations: 0.000000
..connectivity..
..matrix multiplication (system BLAS)..
..normalization..
..done.
